**Yelp Online Review Classification: Identifying Positive, Negative, and Neutral Comments on Food and Restraunt Quality**


1. Positive: Reviews discussing good service or tasty food.
2. Negative: Reviews expressing dissatisfaction with dirty conditions or tasteless food.
3. Neutral: Comments addressing non-related issues not specifically related to food taste or conditions.

Dataset: The data span a period of more than 15 years, including all ~20,000 reviews. There are almost 45 restaurant reviews.

Expected Outcomes: By the end of this notebook, we aim to have a trained and validated model capable of accurately classifying user reviews into the specified categories. This information will provide valuable insights into the overall food quality perception of McDonald's (US) stores and help identify areas for improvement.

Note: The code and methodologies presented in this notebook can be adapted for other analyzing any sort of bias in the given data


In [1]:
import pandas as pd
import numpy as np

import warnings

warnings.filterwarnings("ignore")

data = pd.read_csv("data/Yelp Restaurant Reviews.csv")
data.rename(columns={"Review Text" : "text"}, inplace=True)
data = data[["text"]]
data['text'] = data['text'].replace(regex='(@\w+)|#|&|!',value='')
data['text'] = data['text'].replace(regex=r'http\S+', value='')

data

,text
0,All I can say is they have very good ice cream...
1,Nice little local place for ice cream.My favor...
2,A delicious treat on a hot day Staff was very ...
3,This was great service and a fun crew I got th...
4,This is one of my favorite places to get ice c...
...,...
19891,Had the chocolate cannoli The filling was rich...
19892,Love apricot croissant I bought it at 4:00 PM....
19893,Line was about 25 people long. It went fast Th...
19894,Its hard not to order everything when I come h...


In [2]:
data = data[data.index.isin(
    [i for i, s in enumerate(data.text) if len(s.split()) < 25]
)]
data.index = np.arange(len(data))
data

,text
0,The best soft serve hands down...no further ex...
1,Much better than Champaign's Jarlings. Better ...
2,Basically the best thing since sliced bread. D...
3,Great service Not your average ice cream stop ...
4,Absolutely awful experience It was hair in my ...
...,...
2147,Excellent chai and ice cream. They also sell g...
2148,Phenomenal flavor selection They make it hard ...
2149,"Delicious ice cream, efficient and kind employ..."
2150,Perfect neighborhood ice cream shop. Great inv...


In [4]:
from src.wordbiases import CalculateWordBias
from src.model import LikelihoodModelForNormalDist

target_set_1 = ["bad", "disappointing", "gross", "awful"]
target_set_2   = ["good", "tasty", "delicious", "amazing"] 
F = ["ADJ", "NOUN", "PRON"]

F = ["ADJ", "NOUN", "PRON"]
wbcalc = CalculateWordBias(target_set_1, target_set_2, F, computing_device="cuda")
wbcalc.process_documents(data, "text")
c1, c2 = wbcalc.calculate_target_embeddings()

wbiases, _ , biased_words = wbcalc.calculate_biases()
total_pop = [b for _, b in biased_words]

mu = np.mean(total_pop)
sigma = np.std(total_pop)

# TODO : Find a way to compute or estimate t1 or t2
# Dividing the populations in 7-93-7
likelihood_clf = LikelihoodModelForNormalDist(0.2, 0.8, threshold_limit=0)
likelihood_clf.fit_total_pop(total_pop)

preds = likelihood_clf.predict(wbiases)[0]
data["prclass"] = preds


100%|██████████| 2152/2152 [00:01<00:00, 2044.36it/s]


In [9]:
likelihood_clf = LikelihoodModelForNormalDist(0.2, 0.8, threshold_limit=0)
likelihood_clf.fit_total_pop(total_pop)

preds = likelihood_clf.predict(wbiases)[0]
data["prclass"] = preds


100%|██████████| 2152/2152 [00:01<00:00, 1811.69it/s]


In [10]:
data

,text,prclass
0,The best soft serve hands down...no further ex...,0
1,Much better than Champaign's Jarlings. Better ...,2
2,Basically the best thing since sliced bread. D...,2
3,Great service Not your average ice cream stop ...,0
4,Absolutely awful experience It was hair in my ...,0
...,...,...
2147,Excellent chai and ice cream. They also sell g...,1
2148,Phenomenal flavor selection They make it hard ...,1
2149,"Delicious ice cream, efficient and kind employ...",2
2150,Perfect neighborhood ice cream shop. Great inv...,1


In [11]:
for s in data[data.prclass == 0].text:
    print(s)

The best soft serve hands down...no further explanation needed Every flavor...those that are constant or those that change weekly...LOVE. THE. BEST.
Great service Not your average ice cream stop but work a stop to try something different. Fun choices for kids and adults
Absolutely awful experience It was hair in my ice cream and the manager was super rude about it and said they don't give refunds
YUMMM I can't wait to try all of the different flavors This place is a must try
Kinda pricey and my one order took 15 minutes. Think I'll just go to Culver's next time
These cupcakes were so pretty looking bit very dry as Hell. The icing was Awesome Too Bad Cupcake Boss but your Cupcakes Sucks
Line too long  the staff too slow. Such a bummer
Very upset that they didn't let's us know ahead of Time that they where out of cannoli 
Italian bakery with no cannoli  tiramisu? What a disappointment. No wonder there's no line waiting in front of the store anymore.
Super over rated, after 40 minutes in 

In [12]:
for s in data[data.prclass == 1].text:
    print(s)

Drinks were quickly made and very tastyy, cured my boba cravings l. The shop was cute inside as well :))
Delicious vegan ice cream and drinks are so aesthetically pleasing The interior is cute and great place to just hangout.
Great service, pretty atmosphere  delicious boba drinks. They're hardworking here and make amazing drinks Totally recommend :)
Ice cream  milk shake were good there are a lot of varieties Staff so not so friendly
Their boba is best Just right. Not too melt or not too hard. And Ice creams are amazing. We love this place.
Delicious high quality drinks Love them They even had amazing hot coco bombs around Christmas time.
Omg matcha mixin with fruity pebbles Very helpful staff I'm going to be back Great atmosphere and music.
It was delicious definitely going back a lot to pick from but it's all yummy and the service was great
Its amazing matcha place  Must visit if in chandler , variety of matcha item on the menu
They have good shakes and good Boba tea's and really fr

In [13]:
for s in data[data.prclass == 2].text:
    print(s)

Much better than Champaign's Jarlings. Better service, better ice cream, and the lines usually significantly shorter. Worth the drive
Basically the best thing since sliced bread. Definitely worth the trip outside of Champaign
Visited from Cali- it's the only boba place I went to in AZ and I miss it Really wish they had one here too
So busy... but worth it DELICIOUS The staff were so nice and gave everyone samples while they waited Only recommendation... a bigger location
Very nice dessert spot. If I didnt live so far I'd be here once a week The Butterfly Lemonade is EXCEPTIONAL
Love the service and the ice cream We also tried the drinks, the butterfly rose lemonade is so good Highly recommend
Loved it Perfect for a late night treat Highly recommend trying out their special shakes
This place is BOMB My husband and I LOVE all their milkshakes. Definitely has turned into one of our go to sweets joint. Perfection
Super cool place. So many dairy free options and a very nice staff I would hi